In [9]:
import time
from collections.abc import MutableSequence

from gammapy.data import Observation
from gammapy.datasets import Dataset
from gammapy.makers import Maker

# Provenance tracking class definition

In [10]:
class Provenance:
    def __init__(self, name=None):
        self.steps = []
        if name:
            self.name = name
        else:
            self.name = f"record-{time.time()}"

    def __enter__(self):
        record_start = time.time()
        self.steps.append((record_start, "start processing"))
        self._start_idx = len(self.steps) - 1

        self.datasets = []
        self.observations = []

    def __exit__(self, exc_type, exc_value, traceback):

        if len(self.datasets) > 0:
            self.steps.append((time.time(), "Datasets:", self.datasets))
        else:
            raise RuntimeError("No dataset is being tracked in this context")

        if len(self.observations) > 0:
            self.steps.append((time.time(), "Observations:", self.observations))
        else:
            raise RuntimeError("No observations are being tracked in this context")
            
        record_stop = time.time()
        record_start = self.steps[self._start_idx][0]
        elapsed = record_stop - record_start
        self.steps.append(
            (record_stop, f"processing finished, {elapsed:.3f} s elaspsed")
        )
        if exc_type:
            print(f"Error during processing: {exc_value}")
            return False  # Don't suppress the exception

    def track(self, target):
        trackable = type(target)
        if issubclass(trackable, list) or issubclass(trackable, MutableSequence):
            for itm in target:
                self.track(itm)
        elif issubclass(trackable, Maker):
            self.add_maker(target)
        elif issubclass(trackable, Observation):
            self.add_observation(target)
        elif issubclass(trackable, Dataset):
            self.add_dataset(target)
        else:
            raise ValueError(f"Cannot track object of type {type(trackable)}")

    def add_maker(self, maker):
        self.steps.append((maker.tag, str(maker)))

    def add_dataset(self, dataset):
        self.datasets.append(
            (
                dataset.tag,
                dataset.name,
            )
        )

    def add_observation(self, obs):
        self.observations.append((obs.obs_id, *observation_provenance_info(obs)))


def observation_provenance_info(obs):
    return (
        obs.meta.obs_info.telescope,
        obs.meta.obs_info.instrument,
        obs.meta.obs_info.observation_mode,
        obs.meta.time_info.time_start,
    )

# Usage example

## Analysis setup

In [11]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore
from gammapy.datasets import Datasets, MapDataset
from gammapy.estimators import ExcessMapEstimator
from gammapy.makers import FoVBackgroundMaker, MapDatasetMaker, SafeMaskMaker
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from regions import CircleSkyRegion

In [12]:
data_store = DataStore.from_dir(f"/Users/tbylund/DATA/GAMMAPY_DATA/dev/hess-dl3-dr1/")

In [13]:
target_position = SkyCoord.from_name("RX J1713.7-3946").galactic
sel_obs = data_store.obs_table.select_sky_circle(
    center=target_position, radius=3 * u.deg
)
observations = data_store.get_observations(sel_obs["OBS_ID"])

In [14]:
energy_axis = MapAxis.from_energy_bounds(0.3, 10.0, 15, unit="TeV")
energy_axis_true = MapAxis.from_energy_bounds(
    0.1, 20, 20, unit="TeV", name="energy_true"
)

geom = WcsGeom.create(
    skydir=target_position,
    binsz=0.02,
    width=(6, 6),
    frame="galactic",
    axes=[energy_axis],
)

In [15]:
stacked = MapDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name="rxj-stacked"
)
circle = CircleSkyRegion(center=target_position, radius=0.7 * u.deg)
regions = [circle]
exclusion_mask = ~geom.region_mask(regions=regions)

In [16]:
maker = MapDatasetMaker(selection=["counts", "background", "psf", "edisp", "exposure"])
safe_mask_maker = SafeMaskMaker(
    methods=["offset-max", "aeff-max", "bkg-peak"], offset_max="2.3 deg"
)
fov_bkg_maker = FoVBackgroundMaker(method="fit", exclusion_mask=exclusion_mask)

## Provenance tracking example

In [17]:
provenance = Provenance()

with provenance:
    provenance.track([maker, safe_mask_maker, fov_bkg_maker])
    provenance.track(observations)
    for obs in observations:
        dataset = maker.run(stacked, obs)
        dataset = safe_mask_maker.run(dataset, obs)
        dataset = fov_bkg_maker.run(dataset)
        stacked.stack(dataset)

    provenance.track(stacked)

In [26]:
dataset._meta

MapDatasetMetaData(creation=CreatorMetaData(creator='Gammapy 2.0.dev2346+ge586a95dd.d20251120', date=<Time object: scale='utc' format='datetime' value=2025-11-20 09:53:09.879283>, origin=None), obs_info=[ObsInfoMetaData(obs_id=20900, telescope='HESS', instrument=None, sub_array=None, observation_mode='POINTING')], pointing=[PointingInfoMetaData(radec_mean=<SkyCoord (ICRS): (ra, dec) in deg
    (258.0191875, -39.34900722)>, altaz_mean=None)], event_type=None, optional=None)

In [27]:
dataset.meta

MapDatasetMetaData(creation=CreatorMetaData(creator='Gammapy 2.0.dev2346+ge586a95dd.d20251120', date=<Time object: scale='utc' format='datetime' value=2025-11-20 09:53:09.879283>, origin=None), obs_info=[ObsInfoMetaData(obs_id=20900, telescope='HESS', instrument=None, sub_array=None, observation_mode='POINTING')], pointing=[PointingInfoMetaData(radec_mean=<SkyCoord (ICRS): (ra, dec) in deg
    (258.0191875, -39.34900722)>, altaz_mean=None)], event_type=None, optional=None)

In [24]:
stacked.meta

MapDatasetMetaData(creation=CreatorMetaData(creator='Gammapy 2.0.dev2346+ge586a95dd.d20251120', date=<Time object: scale='utc' format='datetime' value=2025-11-20 09:52:37.331701>, origin=None), obs_info=None, pointing=None, event_type=None, optional=None)

In [80]:
provenance.steps

[(1762783787.977553, 'start processing'),
 ('MapDatasetMaker',
  "MapDatasetMaker\n---------------\n\n  selection                      : {'edisp', 'counts', 'exposure', 'background', 'psf'}\n  background_interp_missing_data : True\n  background_pad_offset          : True\n  fov_rotation_step              : 1.0 deg\n"),
 ('SafeMaskMaker',
  "SafeMaskMaker\n-------------\n\n  methods      : {'aeff-max', 'bkg-peak', 'offset-max'}\n  aeff_percent : 10\n  bias_percent : 10\n  offset_max   : 2.3 deg\n  irfs         : DL4\n"),
 ('FoVBackgroundMaker',
  "FoVBackgroundMaker\n------------------\n\n  method               : fit\n  exclusion_mask       : WcsNDMap\n\n  geom  : WcsGeom \n  axes  : ['lon', 'lat', 'energy']\n  shape : (300, 300, 15)\n  ndim  : 3\n  unit  : \n  dtype : bool\n\n  min_counts           : 0\n  min_npred_background : 0\n  fit                  : <gammapy.modeling.fit.Fit object at 0x158763050>\n"),
 (1762783812.067244, 'Datasets:', [('MapDataset', 'rxj-stacked')]),
 (17627838